In [ ]:
!pip install torch torchvision matplotlib


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils.prune as prune
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, TensorDataset
from sklearn.model_selection import KFold
import numpy as np
import matplotlib.pyplot as plt
import time


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        return self.fc2(x)


In [ ]:
def train(model, train_loader, optimizer, criterion, device):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

def evaluate(model, data_loader, device):
    model.eval()
    correct = 0
    total = 0
    start_time = time.time()
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    end_time = time.time()
    return 100 * correct / total, end_time - start_time


In [ ]:
def add_noise(dataset, noise_level=0.3):
    raw = dataset.data.float() / 255.0
    noise = torch.randn_like(raw) * noise_level
    noisy = torch.clamp(raw + noise, 0., 1.)
    noisy = (noisy - 0.1307) / 0.3081
    return TensorDataset(noisy.unsqueeze(1), dataset.targets)


In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
clean_test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)
noisy_test_loader = DataLoader(add_noise(test_dataset, 0.3), batch_size=1000)


100%|██████████| 9.91M/9.91M [00:00<00:00, 132MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 29.2MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 111MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.19MB/s]


In [ ]:
def apply_pruning(model, method, amount=0.4):
    model = model.to('cpu')
    parameters_to_prune = [
        (model.conv1, 'weight'),
        (model.conv2, 'weight'),
        (model.fc1, 'weight'),
        (model.fc2, 'weight')
    ]

    if method == "random_unstructured":
        for module, param in parameters_to_prune:
            prune.random_unstructured(module, name=param, amount=amount)

    elif method == "l1_unstructured":
        for module, param in parameters_to_prune:
            prune.l1_unstructured(module, name=param, amount=amount)

    elif method == "random_structured":
        for module, param in parameters_to_prune:
            prune.random_structured(module, name=param, amount=0.4, dim=0)

    elif method == "ln_structured":
        for module, param in parameters_to_prune:
            prune.ln_structured(module, name=param, amount=0.4, n=2, dim=0)

    elif method == "global_unstructured":
        prune.global_unstructured(parameters_to_prune, pruning_method=prune.L1Unstructured, amount=amount)

    elif method == "identity":
        for module, param in parameters_to_prune:
            prune.identity(module, param)

    elif method == "custom_mask":
        for module, param in parameters_to_prune:
            prune.custom_from_mask(module, name=param, mask=torch.bernoulli(torch.full_like(module.weight, 0.6)))

    return model


In [ ]:
kf = KFold(n_splits=3, shuffle=True, random_state=42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pruning_methods = [
    'random_unstructured', 'l1_unstructured', 'random_structured',
    'ln_structured', 'global_unstructured', 'identity', 'custom_mask'
]

results = []

for method in pruning_methods:
    print(f"\n=== {method.upper()} ===")
    fold_accuracies = []

    for fold, (train_idx, _) in enumerate(kf.split(dataset)):
        model = CNN().to(device)
        model = apply_pruning(model, method)

        train_loader = DataLoader(Subset(dataset, train_idx), batch_size=64, shuffle=True)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss()

        for epoch in range(2):  # You can increase to 5+ for better accuracy
            train(model, train_loader, optimizer, criterion, device)

        acc_clean, time_clean = evaluate(model, clean_test_loader, device)
        acc_noisy, time_noisy = evaluate(model, noisy_test_loader, device)
        fold_accuracies.append((acc_clean, time_clean, acc_noisy, time_noisy))
        print(f"Fold {fold+1}: Clean={acc_clean:.2f}%, Noisy={acc_noisy:.2f}%")

    avg = np.mean(fold_accuracies, axis=0)
    results.append((method, *avg))



=== RANDOM_UNSTRUCTURED ===
Fold 1: Clean=98.50%, Noisy=95.42%
Fold 2: Clean=98.44%, Noisy=95.40%
Fold 3: Clean=98.40%, Noisy=94.83%

=== L1_UNSTRUCTURED ===
Fold 1: Clean=98.40%, Noisy=94.86%
Fold 2: Clean=98.44%, Noisy=95.06%
Fold 3: Clean=98.26%, Noisy=93.85%

=== RANDOM_STRUCTURED ===
Fold 1: Clean=69.82%, Noisy=68.40%
Fold 2: Clean=69.34%, Noisy=64.02%
Fold 3: Clean=68.66%, Noisy=67.04%

=== LN_STRUCTURED ===
Fold 1: Clean=70.43%, Noisy=68.07%
Fold 2: Clean=69.08%, Noisy=66.20%
Fold 3: Clean=69.29%, Noisy=65.07%

=== GLOBAL_UNSTRUCTURED ===
Fold 1: Clean=98.58%, Noisy=96.82%
Fold 2: Clean=98.71%, Noisy=96.65%
Fold 3: Clean=98.45%, Noisy=93.15%

=== IDENTITY ===
Fold 1: Clean=98.65%, Noisy=93.71%
Fold 2: Clean=98.66%, Noisy=96.63%
Fold 3: Clean=98.22%, Noisy=94.12%

=== CUSTOM_MASK ===
Fold 1: Clean=97.72%, Noisy=73.18%
Fold 2: Clean=98.16%, Noisy=94.54%
Fold 3: Clean=98.19%, Noisy=94.82%


In [ ]:
print("\n===== Summary =====")
print("Method\t\t\tClean Acc\tTime\tNoisy Acc\tTime")
for row in results:
    print(f"{row[0]:<20} {row[1]:.2f}%\t{row[2]:.2f}s\t{row[3]:.2f}%\t{row[4]:.2f}s")



===== Summary =====
Method			Clean Acc	Time	Noisy Acc	Time
random_unstructured  98.45%	7.85s	95.22%	6.37s
l1_unstructured      98.37%	8.78s	94.59%	7.19s
random_structured    69.27%	7.81s	66.49%	5.44s
ln_structured        69.60%	7.30s	66.45%	6.22s
global_unstructured  98.58%	8.03s	95.54%	6.06s
identity             98.51%	7.87s	94.82%	6.44s
custom_mask          98.02%	7.86s	87.51%	6.27s
